# Terrain Classification - Combined User Data
### Created by Keenan McConkey 2019.08.01

In [1]:
from __future__ import absolute_import, division, print_function

import pandas as pd

import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D
from scipy import signal
from scipy import stats

from datetime import datetime
from decimal import Decimal

import pymrmr
import sklearn

## Part 1 - Importing Preprocessed Data

### Part (a) - Functions for Data Import

In [2]:
# All the terrains, placements, vectors, power-assistance, users in the study
terrains = ['Concrete', 'Carpet', 'Linoleum', 'Asphalt', 'Sidewalk', 'Grass', 'Gravel']
powers = ['Manual'] # TODO: Fix power PSD data and add back in
placements_manual = ['Middle', 'Left', 'Right', 'Synthesis']
placements_power = ['Middle']
vectors = ['Features', 'FFTs', 'PSDLogs']
users = ['Keenan', 'Kevin', 'Mahsa', 'Jamie']
axes = ['X Accel', 'Y Accel', 'Z Accel', 'X Gyro', 'Y Gyro', 'Z Gyro']

In [3]:
'''Get the integer terrain value of a given label'''
def get_terrain_num(_label):
    for i, terrain in enumerate(terrains):
        if terrain in _label:
            return i
        
    raise Exception('Unknown terrain')

'''Get the name associated with a terrain integer'''
def get_terrain_name(terrain_num):
    return terrains[terrain_num]
    return terrains[terrain_num]

'''Get the placement location name for given label'''
def get_placement(_label):
    for placement in placements:
        if placement in _label:
            return placement
    
    raise Exception('Unknown placement')

'''Get the transform used for given label'''
def get_transform(_label):
    for transform in transforms:
        if transform in _label:
            return transform
    
    raise Exception('Unknown transform')

### Part (b) - Import Processed Data from Each User

In [4]:
'''Combine data from labelled datasets into a single dataframe'''
def combine_datasets(datasets):
    return pd.concat(list(datasets.values()), ignore_index=True, sort=False)

In [5]:
path = 'processed_data/new_setup/' 

# Nested dictionary of processed data:
# - Power assistance type
# -- Placement
# --- Feature Vector
# ---- User
power_dict = {}

# Create each nesting of the dictionary
for power in powers:
    placement_dict = {}
    
    # Power datasets only have middle placement (for now)
    if power == 'Power':
        placements = placements_power.copy()
    else:
        placements = placements_manual.copy()
    
    for placement in placements:
        vector_dict = {}

        for vector in vectors:
            user_dict = {}

            for user in users:
                # File name based on above parameters
                filename = power.lower() + '/' + placement + '_' + vector + '_Filt_' + user 
                if power == 'Power':
                    filename += '_Power'
                filename += '.csv'
                
                # Read data and update current user dictionary
                data = pd.read_csv(path + filename)
                user_dict.update({user: data})

            # Combine users to form a new entry of user dictionary, save to .csv
            # NaNs arise when you combine Synthesis feature vectors
            combined_data = combine_datasets(user_dict).dropna(axis='columns')
            user_dict.update({'All': combined_data})

            vector_dict.update({vector: user_dict})
        
        # Create a dictionary of the combined feature vector for each user
        combined_vector_user_dict = {}
        
        for user in user_dict.keys():
            # Get all vectors for current user and pop label column
            user_all_vectors = []
            
            for vector in vector_dict.values():
                user_vector = vector[user].copy()
                labels = user_vector.pop('Label') # All label columns should be the same
                user_all_vectors.append(user_vector)
            
            # Combine vectors and add back label column
            combined_vector = pd.concat(user_all_vectors, axis='columns')
            combined_vector.insert(loc=0, column='Label', value=labels)
            combined_vector_user_dict.update({user: combined_vector})
        
        # Add the combined feature vector to the vector dictionary
        vector_dict.update({'Combined': combined_vector_user_dict})
        
        placement_dict.update({placement: vector_dict})
    
    power_dict.update({power: placement_dict})

In [6]:
# Update vectors to reflect new combined vector
vectors.append('Combined')

In [7]:
# Check some data
power_dict['Manual']['Middle']['Combined']['All'].tail()

,Label,Mean X Accel Middle,Std Dev X Accel Middle,L2 Norm X Accel Middle,Autocorrelation X Accel Middle,Max X Accel Middle,Min X Accel Middle,Root Mean Squared X Accel Middle,Zero Crossing Rate X Accel Middle,Skew X Accel Middle,...,PSD 1.0 Hz X Gyro Middle,PSD 1.0 Hz Z Accel Middle,PSD 1.0 Hz Y Accel Middle,PSD 1.0 Hz X Accel Middle,PSD 0.0 Hz Z Gyro Middle,PSD 0.0 Hz Y Gyro Middle,PSD 0.0 Hz X Gyro Middle,PSD 0.0 Hz Z Accel Middle,PSD 0.0 Hz Y Accel Middle,PSD 0.0 Hz X Accel Middle
8012,5,-0.048488,-0.146781,-0.224684,-0.293490,-0.369834,0.803903,-0.224684,0.410072,-0.295411,...,-3.199153,-2.119317,-1.360919,-1.726940,-1.125372,-2.414256,-3.104088,-1.992795,-0.906233,-0.163503
8013,5,-0.838211,-0.549810,-0.679571,-0.697007,-0.727631,0.649344,-0.679571,-1.279923,0.891646,...,-2.854475,-1.476822,-1.801745,-0.448959,-1.699782,-2.961492,-2.498364,-1.379586,-1.488251,0.043019
8014,5,-0.058510,-1.523523,-1.527153,-1.358674,-0.612425,0.753509,-1.527153,1.255070,0.353343,...,-2.651156,-2.436379,-1.318068,-1.271694,-1.236619,-2.188757,-2.502940,-1.908180,-1.126076,-0.427966
8015,5,0.068267,1.145593,1.019167,0.982532,2.396203,-0.863990,1.019167,-0.434925,1.624021,...,-3.348061,-2.113860,-2.132868,-0.466117,-1.621375,-2.014946,-3.373160,-2.914507,-1.887899,0.042225
8016,5,-0.180351,-1.198178,-1.241871,-1.149074,-0.756011,1.430402,-1.241871,-0.857424,-0.327254,...,-3.855377,-2.770625,-2.242923,-1.473848,-1.418449,-2.783864,-2.749813,-2.456584,-0.994285,-0.041851


## Part 2 -  mRMR (minimum Redunancy Maximum Relevance)

mRMR tries to find which features have the highest information shared with classified state and lowest information shared with other features.

### Part (a) - Middle Frame Placement

#### Part (i) - Manual Wheelchair

In [8]:
pymrmr.mRMR(data=power_dict['Manual']['Middle']['Features']['All'], method='MID', nfeats=5)

['Variance Frequency X Accel Middle',
 'Excess Kurtosis Z Accel Middle',
 'Variance Frequency Y Gyro Middle',
 'Zero Crossing Rate Z Gyro Middle',
 'Variance Frequency X Gyro Middle']

In [9]:
pymrmr.mRMR(data=power_dict['Manual']['Middle']['FFTs']['All'], method='MID', nfeats=5)

['FFT 0.0 Hz Y Accel Middle',
 'FFT 32.0 Hz Z Accel Middle',
 'FFT 31.0 Hz Z Accel Middle',
 'FFT 29.0 Hz Z Accel Middle',
 'FFT 0.0 Hz Z Gyro Middle']

In [10]:
pymrmr.mRMR(data=power_dict['Manual']['Middle']['PSDLogs']['All'], method='MID', nfeats=5)

['PSD 55.0 Hz Y Accel Middle',
 'PSD 0.0 Hz Y Gyro Middle',
 'PSD 37.0 Hz Y Gyro Middle',
 'PSD 3.0 Hz Y Gyro Middle',
 'PSD 56.0 Hz Y Accel Middle']

In [11]:
pymrmr.mRMR(data=power_dict['Manual']['Middle']['Combined']['All'], method='MID', nfeats=5)

['PSD 55.0 Hz Y Accel Middle',
 'PSD 0.0 Hz Y Gyro Middle',
 'PSD 37.0 Hz Y Gyro Middle',
 'PSD 3.0 Hz Y Gyro Middle',
 'PSD 56.0 Hz Y Accel Middle']

#### Part (i) - Power Assist Wheelchair

In [12]:
#pymrmr.mRMR(data=power_dict['Power']['Middle']['Features']['All'], method='MID', nfeats=5)

In [13]:
#pymrmr.mRMR(data=power_dict['Power']['Middle']['FFTs']['All'], method='MID', nfeats=5)

In [14]:
#pymrmr.mRMR(data=power_dict['Power']['Middle']['PSDLogs']['All'], method='MID', nfeats=5)

In [15]:
#pymrmr.mRMR(data=power_dict['Power']['Middle']['Combined']['All'], method='MID', nfeats=5)

### Part (b) - Left Wheel Placement

In [16]:
pymrmr.mRMR(data=power_dict['Manual']['Left']['Features']['All'], method='MID', nfeats=5)

['Min Y Accel Left',
 'Zero Crossing Rate Y Gyro Left',
 'Min X Accel Left',
 'Max X Accel Left',
 'Max Y Accel Left']

In [17]:
pymrmr.mRMR(data=power_dict['Manual']['Left']['FFTs']['All'], method='MID', nfeats=5)

['FFT 0.0 Hz Z Gyro Left',
 'FFT 33.0 Hz X Accel Left',
 'FFT 4.0 Hz Y Accel Left',
 'FFT 52.0 Hz Z Accel Left',
 'FFT 4.0 Hz X Accel Left']

In [18]:
pymrmr.mRMR(data=power_dict['Manual']['Left']['PSDLogs']['All'], method='MID', nfeats=5)

['PSD 63.0 Hz Z Accel Left',
 'PSD 16.0 Hz Z Accel Left',
 'PSD 36.0 Hz Z Accel Left',
 'PSD 47.0 Hz X Accel Left',
 'PSD 53.0 Hz Y Accel Left']

In [19]:
pymrmr.mRMR(data=power_dict['Manual']['Left']['Combined']['All'], method='MID', nfeats=5)

['PSD 63.0 Hz Z Accel Left',
 'PSD 16.0 Hz Z Accel Left',
 'PSD 36.0 Hz Z Accel Left',
 'FFT 0.0 Hz Z Gyro Left',
 'PSD 27.0 Hz Z Accel Left']

### Part (c) - Right Wheel Placement

In [20]:
pymrmr.mRMR(data=power_dict['Manual']['Right']['Features']['All'], method='MID', nfeats=5)

['Min X Accel Right',
 'Min Y Accel Right',
 'Variance Frequency Z Accel Right',
 'Max X Accel Right',
 'Max Y Accel Right']

In [21]:
pymrmr.mRMR(data=power_dict['Manual']['Right']['FFTs']['All'], method='MID', nfeats=5)

['FFT 0.0 Hz Z Gyro Right',
 'FFT 53.0 Hz Z Accel Right',
 'FFT 52.0 Hz Z Accel Right',
 'FFT 55.0 Hz Z Accel Right',
 'FFT 54.0 Hz Z Accel Right']

In [22]:
pymrmr.mRMR(data=power_dict['Manual']['Right']['PSDLogs']['All'], method='MID', nfeats=5)

['PSD 62.0 Hz Z Accel Right',
 'PSD 16.0 Hz Z Accel Right',
 'PSD 56.0 Hz Z Accel Right',
 'PSD 46.0 Hz X Accel Right',
 'PSD 20.0 Hz Z Accel Right']

In [23]:
pymrmr.mRMR(data=power_dict['Manual']['Right']['Combined']['All'], method='MID', nfeats=5)

['PSD 62.0 Hz Z Accel Right',
 'PSD 16.0 Hz Z Accel Right',
 'PSD 56.0 Hz Z Accel Right',
 'FFT 0.0 Hz Z Gyro Right',
 'PSD 63.0 Hz Z Accel Right']

### Part (d) - Synthesis "Placement"

In [24]:
pymrmr.mRMR(data=power_dict['Manual']['Synthesis']['Features']['All'], method='MID', nfeats=5)

['Min Left Y Accel Synthesis',
 'Min Right X Accel Synthesis',
 'Zero Crossing Rate Left Y Gyro Synthesis',
 'Min Left X Accel Synthesis',
 'Min Right Y Accel Synthesis']

In [25]:
pymrmr.mRMR(data=power_dict['Manual']['Synthesis']['FFTs']['All'], method='MID', nfeats=5)

['FFT 0.0 Hz Right Z Gyro Synthesis',
 'FFT 0.0 Hz Left XY Accel Synthesis',
 'FFT 0.0 Hz Calc Z Gyro Synthesis',
 'FFT 0.0 Hz Right XY Accel Synthesis',
 'FFT 0.0 Hz Left Z Gyro Synthesis']

In [26]:
pymrmr.mRMR(data=power_dict['Manual']['Synthesis']['PSDLogs']['All'], method='MID', nfeats=5)

['PSD 39.0 Hz Left XY Accel Synthesis',
 'PSD 15.0 Hz Right Z Accel Synthesis',
 'PSD 63.0 Hz Right Z Accel Synthesis',
 'PSD 51.0 Hz Left XY Accel Synthesis',
 'PSD 13.0 Hz Left XY Accel Synthesis']

In [27]:
pymrmr.mRMR(data=power_dict['Manual']['Synthesis']['Combined']['All'], method='MID', nfeats=5)

['PSD 39.0 Hz Left XY Accel Synthesis',
 'PSD 15.0 Hz Right Z Accel Synthesis',
 'PSD 63.0 Hz Right Z Accel Synthesis',
 'PSD 51.0 Hz Left XY Accel Synthesis',
 'PSD 13.0 Hz Left XY Accel Synthesis']

### Part (e) - Nested Dictionary of Top Features

In [ ]:
# Create dictionary of top mRMR features to speed up calculations, up to to 20
power_dict_features = {}

for power in powers:
    placement_dict_features = {}
    
    for placement in placements:
        vector_dict_features = {}
        
        for vector in vectors:
            user_dict_features = {}

            # Only calculate for all users for now
            for user in ['All']:
                top_features = pymrmr.mRMR(data=power_dict[power][placement][vector][user],
                                           method='MID', nfeats=20)
                user_dict_features.update({user: top_features})

            vector_dict_features.update({vector: user_dict_features})
            
        placement_dict_features.update({placement: vector_dict_features})
    
    power_dict_features.update({power: placement_dict_features})

In [ ]:
# Pickle the dictionary
import pickle

mRMR_dict_filename = '/dicts/mRMR_Dictionary.pkl'
outfile = open(mRMR_dict_filename, 'wb')
pickle.dump(power_dict_features, outfile)
outfile.close()

In [ ]:
# Unpickle the dictionary
#infile = open(mRMR_dict_filename, 'rb')
#power_dict_features = pickle.load(infile)
#infile.close()

## Part 3 - PCA (Principal Component Analysis)

In [ ]:
from sklearn.decomposition import PCA

def get_pca_df(combined_data, n_components=2):
    # Setup PCA parameters
    pca = PCA(n_components=n_components)
    
    # Copy data to avoid modification
    data = combined_data.copy()
    
    # Extract terrain labels
    labels = data.pop('Label')
    
    # Get specified number of principal components and convert to dataframe
    pc = pca.fit_transform(data)
    pc_df = pd.DataFrame(data=pc, columns=['PC {}'.format(i + 1) for i in range(n_components)])
    
    print('Explained Variance of Each PC: {}'.format(pca.explained_variance_ratio_))
    print('Total Explained Variance: {}'.format(np.sum(pca.explained_variance_ratio_)))
    
    return pd.concat([labels, pc_df], axis='columns')

In [ ]:
get_pca_df(power_dict['Manual']['Synthesis']['Combined']['All']).tail()

In [ ]:
get_pca_df(power_dict['Manual']['Synthesis']['Combined']['All'], n_components=0.85).tail()

In [ ]:
def visualize_2d_pca(pca_2d_df, figsize=(8, 8)):
    # Plot parameters
    plt.clf()
    plt.figure(figsize=figsize)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    
    # Scatter plot of each terrain
    for terrain in terrains:
        terrain_indices = pca_2d_df['Label'] == get_terrain_num(terrain)
        plt.scatter(pca_2d_df.loc[terrain_indices, 'PC 1'], pca_2d_df.loc[terrain_indices, 'PC 2'])
    
    plt.legend(terrains)
    plt.show()

In [ ]:
visualize_2d_pca(get_pca_df(power_dict['Manual']['Right']['Features']['All']))

## Part 4 - Training Classifiers

In [ ]:
from sklearn.model_selection import KFold

'''Run train test k-fold times
   Returns an tuple of arrays, where arrays elements are actual/predicted labels for each k-fold test'''
def train_test_k_fold(combined_data, n_splits, model):
    # Shuffle ensures we get a mix of terrains
    kf = KFold(n_splits=n_splits, shuffle=True)

    # Copy data to avoid modification
    data = combined_data.copy()
    
    # Extract terrain labels
    labels = data.pop('Label')

    # Array of predicted labels for each k fold
    test_k_fold = []
    predict_k_fold = []

    # Split into n splits
    for train_index, test_index in kf.split(data):
        train, test = data.loc[train_index], data.loc[test_index]
        train_labels, test_labels = labels.loc[train_index], labels.loc[test_index]
        
        # Actual labels
        test_k_fold.append(test_labels)
        
        # Train and test model
        model.fit(train, train_labels)
        predict_k_fold.append(model.predict(test))

    return (test_k_fold, predict_k_fold)

### Part (a) - Create Accuracy Table

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Dictionary of classifiers
classifiers = {'Naive Bayes': GaussianNB(),
               'k Nearest': KNeighborsClassifier(),
               'Decision Tree': DecisionTreeClassifier(), 
               'Random Forest': RandomForestClassifier(n_estimators=100),
               'AdaBoost': AdaBoostClassifier(),
               'Support Vector Machine': SVC(gamma='scale')}

In [ ]:
def create_accuracy_table(n_splits, power_type='Manual', user_name='All', separate_axes=False
                          feat_selection='None', n_feats=None):
    
    # Power type affects which placements are available
    if power_type == 'Manual':
        placements = placements_manual
    else:
        placements = placements_power
    
    vector_indices = [p + ' ' + v for p in placements for v in vectors]
    accuracy_table = pd.DataFrame({'Vector': vector_indices})

    # Calculate accuracy for each placement for each feature vector and classifier
    for classifier_name, classifier in classifiers.items():
        model = classifier

        # Row dictionary for given model
        rows = {}

        # Add current classifier to row dictionary
        for placement in placements:
            for vector in vectors:
                # Extract data for above parameters
                index_name = placement + ' ' + vector
                data = power_dict[power_type][placement][vector][user_name]
                
                # Use only the top features using mRMR feature selection
                if feat_selection == 'mRMR':
                    top_feats = power_dict_features[power_type][placement][vector][user_name]
                    top_feats.insert(0, 'Label')
                    data = data[top_feats[:n_feats+1]]
                
                # Run PCA on the data
                elif feat_selection == 'PCA':
                    data = get_pca_df(data, n_components=n_feats)
                
                # Extract predicted and actual labels for requested user
                actual, predict = train_test_k_fold(data, n_splits, model)

                # Take mean accuracy of k fold testing
                accuracies = []
                for i in range(len(predict)):
                    accuracies.append(accuracy_score(actual[i], predict[i]))
                rows.update({index_name: np.mean(accuracies)})

        # Update accuracy table with classifier column by mapping row names to indices
        accuracy_table[classifier_name] = accuracy_table['Vector'].map(rows)
    
    return accuracy_table

In [ ]:
def create_accuracy_table(n_splits, power_type='Manual', user_name='All', separate_axes=False
                          feat_selection='None', n_feats=None):
    
    # Power type affects which placements are available
    if power_type == 'Manual':
        placements = placements_manual
    else:
        placements = placements_power
    
    # Dataframe table of accuracies for each classifier for each placement
    if separate_axes:
        vector_indices = [p + ' ' + v + ' ' + a for p in placements for v in vectors for a in axes]
    else:
        vector_indices = [p + ' ' + v for p in placements for v in vectors]
    accuracy_table = pd.DataFrame({'Vector': vector_indices})

    # Calculate accuracy for each placement for each feature vector and classifier
    for classifier_name, classifier in classifiers.items():
        model = classifier

        # Row dictionary for given model
        rows = {}

        # Add current classifier to row dictionary
        for placement in placements:
            for vector in vectors:
                # Extract data for above parameters
                data = power_dict[power_type][placement][vector][user_name]
                
                # Iterate through axes if we want them separate, else just put in empty string
                if separate_axes:
                    itr_axes = axes
                else:
                    itr_axes = ['']
                
                for axis in itr_axes:
                    index_name = placement + ' ' + vector
                    
                    if axis != ''
                        index_name += axis
                        matching_columns = [column for column in data.columns if axis in column]
                        matching_columns.insert(0, 'Label')
                        data = data[matching_columns]
                    
                    # Use only the top features using mRMR feature selection
                    if feat_selection == 'mRMR':
                        top_feats = pymrmr.mRMR(data=data, method='MID', nfeats=n_feats)
                        top_feats.insert(0, 'Label')
                        data = data[top_feats]

                    # Run PCA on the data
                    elif feat_selection == 'PCA':
                        data = get_pca_df(data, n_components=n_feats)

                    # Extract predicted and actual labels for requested user
                    actual, predict = train_test_k_fold(data, n_splits, model)

                    # Take mean accuracy of k fold testing
                    accuracies = []
                    for i in range(len(predict)):
                        accuracies.append(accuracy_score(actual[i], predict[i]))
                    
                    rows.update({index_name: np.mean(accuracies)})

        # Update accuracy table with classifier column by mapping row names to indices
        accuracy_table[classifier_name] = accuracy_table['Vector'].map(rows)
    
    return accuracy_table

In [ ]:
# Create accuracy table for 5 k-fold splits without any feature selection
accuracy_table = create_accuracy_table(n_splits=5, power_type='Manual', user_name='All')

In [ ]:
accuracy_table

In [ ]:
# Create accuracy table for 5 k-fold splits with mRMR feature selection
accuracy_table_mRMR = create_accuracy_table(n_splits=5, power_type='Manual', user_name='All',
                                            feat_selection='mRMR', n_feats=20)

In [ ]:
accuracy_table_mRMR

In [ ]:
# Create accuracy table for 5 k-fold splits with mRMR feature selection
accuracy_table_PCA = create_accuracy_table(n_splits=5, power_type='Manual', user_name='All',
                                            feat_selection='PCA', n_feats=20)

In [ ]:
accuracy_table_PCA

### Glossary

`Dataset` - Batch of data recorded on one terrain type

`Data Window` - Split up portion of a `Dataset`

`Direction / Axes` - Linear acceleration or gyroscope in $x,y$ or $z$

`Feature Vector` - Any feature of the data that can be used to classify terrain, e.g. Z Accel Mean, Y Accel FFT, etc

`Extracted Feature Vector` - Features that aren't from transforms, e.g. Z Accel Min, Y Accel Autocorrelation, etc

`Placement` - One of three IMU placements on the wheelchair, i.e. Middle, Left, or Right